In [28]:
!pip install xgboost

import cv2
import numpy as np
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [29]:
#  CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train, y_test = y_train.flatten(), y_test.flatten()

In [30]:
#  Feature Extraction
def extract_features(image):
    features = []

    # Renk histogramı (HSV + RGB)
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hist_hsv = cv2.calcHist([hsv], [0,1,2], None, [8,8,8], [0,180,0,256,0,256])
    hist_hsv = cv2.normalize(hist_hsv, hist_hsv).flatten()

    hist_rgb = cv2.calcHist([image], [0,1,2], None, [8,8,8], [0,256,0,256,0,256])
    hist_rgb = cv2.normalize(hist_rgb, hist_rgb).flatten()

    features.extend(hist_hsv)
    features.extend(hist_rgb)

    # Kenar ve gradient
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    edge_density = np.sum(edges > 0) / edges.size
    features.append(edge_density)

    # Sobel gradient
    grad_x = cv2.Sobel(gray, cv2.CV_64F,1,0,ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F,0,1,ksize=3)
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2).mean()
    features.append(grad_magnitude)

    #  HOG
    hog = cv2.HOGDescriptor(_winSize=(32,32),
                            _blockSize=(16,16),
                            _blockStride=(8,8),
                            _cellSize=(8,8),
                            _nbins=9)
    h = hog.compute(image)
    features.extend(h.flatten())

    return np.array(features)

In [31]:
features, labels = [], []
for i in range(5000):
    features.append(extract_features(x_train[i]))
    labels.append(y_train[i])

features = np.array(features)
labels = np.array(labels)

In [32]:
X_train, X_val, y_train_split, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

In [33]:
#XGBoost
clf = XGBClassifier(n_estimators=300, max_depth=6, learning_rate=0.1, use_label_encoder=False, eval_metric='mlogloss')
clf.fit(X_train, y_train_split)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [10:12:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [34]:
#  Tahmin
y_pred = clf.predict(X_val)

In [35]:
acc = accuracy_score(y_val, y_pred)
print("Doğruluk (Accuracy):", acc)
print(classification_report(y_val, y_pred))

Doğruluk (Accuracy): 0.594
              precision    recall  f1-score   support

           0       0.64      0.58      0.61       105
           1       0.70      0.71      0.70        95
           2       0.52      0.45      0.48        92
           3       0.45      0.46      0.46        95
           4       0.57      0.58      0.57       103
           5       0.53      0.45      0.49       108
           6       0.61      0.67      0.64       108
           7       0.54      0.65      0.59        77
           8       0.62      0.65      0.64       109
           9       0.73      0.73      0.73       108

    accuracy                           0.59      1000
   macro avg       0.59      0.59      0.59      1000
weighted avg       0.59      0.59      0.59      1000

